In [2]:
import pandas as pd
import numpy as np
import nltk
import random
import time
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from copy import copy, deepcopy
import collections
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from pandas.tseries.offsets import *
import re
import collections
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import svm
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
data_ads = "/Users/MaxTan/Documents/CU_16fall/BDA/Project/Data/"

In [4]:
user_raw = pd.read_csv(data_ads + "wow3_user2.csv", names = ["user_id", "review_count", "average_stars", 
                                               "friends", "fans", "votes_cool", "votes_funny", 
                                               "votes_useful", "num", "degree", "coefficient"])


In [5]:
business_raw = pd.read_csv(data_ads+"wow3_business_mysql.csv", names = ["business_id", "name", "latitude", 
                                                           "longitude", "stars", "categories", 
                                                           "review_count", "open"])

In [6]:
review_raw = pd.read_csv(data_ads+"wow3_review_mysql.csv", names = ["review_id", "user_id", "business_id", 
                                                       "stars", "text", "date", "votes_funny", 
                                                       "votes_useful"])

/Users/MaxTan/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
all_raw = pd.read_csv(data_ads+"wow3_all2.csv", names = ["business_id", 
                                                       "name", "stars_business","business_review_count",
                                                       "categories", "open", "review_id", 
                                                       "stars_review", "text", "date", "votes_funny", 
                                                       "votes_useful","user_id", "user_review_count", 
                                                       "average_stars","friends", "fans", "num", 
                                                       "degree", "coefficient"])


In [8]:
#2.Topic Model by Key Word:

In [10]:
#The following codes take 20 minutes to generate a new csv file, which I've store it in the same folder:

bus_gb_df = deepcopy(all_raw)
#Group by different businesses for different rates:
bus_df = bus_gb_df.groupby(['business_id']).mean()
bus_df['business_id'] = bus_df.index
bus_df.index = range(len(bus_df))
bus = list(set(bus_df['business_id']))
ind = [True if i in bus else False for i in business_raw['business_id']]
business_revised = business_raw[ind]

#Merge two tables:
lda_df = pd.merge(bus_df,business_revised,on='business_id')

start_time = time.time()
bus_topic = dict()

for i in range(len(all_raw)):
    ly = all_raw.text[i].lower()
    ly = ly.decode('utf-8')
    
    #1.Tokenization:
    tokens = nltk.word_tokenize(ly)
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]

    tokens = word_stop
    
    
    temp_bus = all_raw.business_id[i]
    if temp_bus not in bus_topic:
        bus_topic[temp_bus] = tokens
    else:
        bus_topic[temp_bus] += tokens



print("--- %s seconds ---" % (time.time() - start_time))

lda_df['tokens'] = pd.Series(['']*len(lda_df),index = lda_df.index)
print len(bus_topic)
for bus in bus_topic:
    #print bus
    tempdf = lda_df[lda_df['business_id']==bus]
    temptext = tempdf['categories'].values[0].lower()[1:-1]+', '.join(bus_topic[bus])
    lda_df.set_value([tempdf.index[0]],'tokens',temptext)

lda_df.to_csv('business_LDA.csv')

--- 2210.07916307 seconds ---
3626


In [18]:
lda_df.tokens[0]

u'bars, nightlifeorder, chicken, wing, look, veri, unappet, especi, pick, one, feather, come, understand, thing, happen, situat, wa, handl, veri, unprofession, sure, server, wa, embarrass, seem, though, avoid, issu, wa, brought, attent, typic, tavern, basic, bar, menu, burger, sandwich, fri, munchi, deep, fri, jalapeno, pretzel, great, chang, usual, fri, chees, stick, usual, order, fri, fish, sandwich, great, size, fill, servic, okay, casual, prompt, main, dine, area, away, bar, nice, quiet, full, pittsburgh, sport, memorabilia, bad, place, look, around, wait, order, love, witti, sign, put, get, think, drive, rt, 88, seem, never, end, construct, zone'

In [19]:
lda_df2 = pd.read_csv('business_LDA.csv')

In [20]:
lda_df2.tokens[0]

'bars, nightlifeorder, chicken, wing, look, veri, unappet, especi, pick, one, feather, come, understand, thing, happen, situat, wa, handl, veri, unprofession, sure, server, wa, embarrass, seem, though, avoid, issu, wa, brought, attent, typic, tavern, basic, bar, menu, burger, sandwich, fri, munchi, deep, fri, jalapeno, pretzel, great, chang, usual, fri, chees, stick, usual, order, fri, fish, sandwich, great, size, fill, servic, okay, casual, prompt, main, dine, area, away, bar, nice, quiet, full, pittsburgh, sport, memorabilia, bad, place, look, around, wait, order, love, witti, sign, put, get, think, drive, rt, 88, seem, never, end, construct, zone'

In [22]:
search_word = 'pizza'

In [24]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from os import path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from wordcloud import WordCloud, STOPWORDS

tokenizer = RegexpTokenizer(r'\w+')
# create English stop words list
en_stop = get_stop_words('en')
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

lda_df2 = pd.read_csv('business_LDA.csv')

/Users/MaxTan/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [ ]:
#1. Sort by stars:
ind = [True if search_word in token else False for token in lda_df2.tokens.values]
lda_df_sub = lda_df2[ind]
lda_df_target = lda_df_sub.sort(['stars_review'],ascending=[0]).iloc[:6]
doc_set = list(lda_df_target.tokens.values)

In [82]:
#2. Sort by number of key word:
lda_df2['count_sw'] = pd.Series([0]*len(lda_df2),index = lda_df2.index)
for i in range(len(lda_df2)):
    temp_value = collections.Counter([item.strip() for item in lda_df2.tokens[i].split(",")])[search_word]
    lda_df2.set_value(i,'count_sw',temp_value)
lda_df_target = lda_df2.sort(['count_sw'],ascending=[0]).iloc[:10]
doc_set = list(lda_df_target.tokens.values)

/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [94]:
start_time = time.time()

for i in range(len(doc_set)):
    texts = []
    
    # clean and tokenize document string
    raw = doc_set[i].lower()
    
    #1.Tokenization:
    tokens = tokenizer.tokenize(raw)
    
    
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, #4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")] 
    
    # add tokens to list
    texts.append(word_stop)

    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=50)
    
    result = ldamodel.print_topics(num_topics=10, num_words=10)
    
    result1 = [j[1] for j in result]
    
    res_topic = []
    
    for item in result1:
        
        res_topic+=str(item).split('"')[1::2]
    
    text = " ".join(res_topic)
    
    top_words = [item[0] for item in collections.Counter(text.split()).most_common(10)]
    print top_words
    
    # Generate a word cloud image
    wordcloud = WordCloud().generate(text)

    # Display the generated image:
    # the matplotlib way:

    plt.figure(figsize=(100,100),dpi =300)
    plt.imshow(wordcloud)
    plt.axis("off")
    
    # take relative word frequencies into account, lower max_font_size
    wordcloud = WordCloud(max_font_size=80,stopwords=STOPWORDS,font_path = 'EraserRegular.ttf',
                          background_color='blue',max_words=100,width=600,height=600).generate(text)
                          #relative_scaling=.8).generate(text)
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(lda_df_target.iloc[i:i+1]['name'].values[0])
    plt.savefig(lda_df_target.iloc[i:i+1]['name'].values[0]+".jpg",figsize=(100,100),dpi=100)
    plt.close()

print("--- %s seconds ---" % (time.time() - start_time))

['great', 'good', 'wa', 'beer', 'thi', 'nt', 'pizza', 'place', 'veri', 'order']
['wa', 'mineo', 'chee', 'place', 'thi', 'nt', 'pizza', 'good', 'pittsburgh', 'like']
['wa', 'gelato', 'place', 'thi', 'flavor', 'nt', 'pizza', 'good', 'like', 'mercurio']
['wa', 'chee', 'thi', 'nt', 'best', 'pizza', 'sauc', 'place', 'fiori', 'good']


/Users/MaxTan/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


['wa', 'thi', 'aiello', 'pizza', 'good', 'place', 'nt', 'chee', 'mineo', 'best']
['good', 'wa', 'food', 'beer', 'place', 'thi', 'church', 'nt', 'great', 'brew']
['good', 'wa', 'place', 'thi', 'nt', 'pizza', 'dinett', 'wine', 'one', 'like']
['good', 'wa', 'nt', 'pizza', 'great', 'porch', 'thi', 'food', 'place', 'order']
['good', 'wa', 'thi', 'pasta', 'pizza', 'wait', 'food', 'nt', 'restaur', 'place']
['wa', 'get', 'beto', 'chee', 'thi', 'nt', 'pizza', 'top', 'like', 'cold']
--- 110.998905897 seconds ---


In [95]:
#Sentimental Analysis(Other Popular Words)


In [112]:
temp_biz = lda_df_target.iloc[i:i+1]['business_id'].values[0]
temp_rev = review_raw[review_raw['business_id']==temp_biz]

In [114]:
set(temp_rev.stars.values)

{'1', '2', '3', '4', '5'}

In [115]:
temp_rev_pos = temp_rev[(temp_rev['stars']==5) | (temp_rev['stars']=='5')]

temp_rev_neg = temp_rev[(temp_rev['stars']<=2) | (temp_rev['stars']=='1')|(temp_rev['stars']=='2')]

In [116]:
print temp_rev_pos.shape,temp_rev_neg.shape

(59, 8) (27, 8)


In [120]:
temp_rev_neg.text.iloc[0]

"There are only so many ways to make the American bastardized version of a pizza, but for some reason there are as many pizza shops as there are stars in a clear night sky. As they may be stunning to behold, from afar each one is pretty much the same thing. This shop is another place that tosses sauce and cheese on bread and serves it to their patrons. Ok, so they DO have a slightly different take, throwing the cold shredded cheese onto the pie as soon as the dough is out of the oven. This does not melt the cheese, and I do see how the pizza would be cold after heaping on the cheese.  Some customers may like it, me, not so much. I tried the Italian hoagie.  It was a good size and the meat was flavorful. But that's about it. They put oil on the sandwich, but no other dressing. I expected a nice tangy flavor to compliment the meat but was disappointed. My date ordered a hot sausage hoagie. I didn't sample it, he liked it, but it was way too messy. The bread got soggy and the darn thing f

In [121]:
#Sentimental Analysis(Random Forest):
review_pos = review_raw[(review_raw['stars']==5) | (review_raw['stars']=='5')]

review_neg = review_raw[(review_raw['stars']<=2) | (review_raw['stars']=='1')|(review_raw['stars']=='2')]

print review_pos.shape
print review_neg.shape

traindf = review_raw[(review_raw['stars']==5) | (review_raw['stars']=='5')|(review_raw['stars']<=2) | (review_raw['stars']=='1')|(review_raw['stars']=='2')]

traindf['sentiment'] = pd.Series([0]*len(traindf),index = traindf.index)

for i in traindf.index:
    star = traindf.loc[i]['stars']
    if star == 5 or star == '5':
        traindf.set_value(i, 'sentiment', 1)

(39691, 8)
(18577, 8)


/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [55]:
"""
#remove those songs that can't be tokenized(have unicode decode error):
start_time  = time.time()

newdf = pd.DataFrame()
for i in traindf.index:
    ly = traindf.loc[i]['text'].lower()
    #ly = ly.translate(None, string.punctuation)
    try:
        tokens = nltk.word_tokenize(ly)
        tokens = [lmtzr.lemmatize(word) for word in tokens]
        tempdf = traindf[i:i+1]
        frames = [newdf,tempdf]
        newdf = pd.concat(frames) 
    except UnicodeDecodeError:
        continue
print len(newdf)
print("--- %s seconds ---" % (time.time() - start_time))
"""

31820
--- 386.350394011 seconds ---


In [122]:
pos_length, neg_length = len(traindf[traindf['sentiment']==1]),len(traindf[traindf['sentiment']==0])
print pos_length, neg_length
trainpos = traindf[traindf['sentiment']==1].iloc[:int(pos_length*0.8)]
trainneg = traindf[traindf['sentiment']==0].iloc[:int(neg_length*0.8)]
testpos = traindf[traindf['sentiment']==1].iloc[int(pos_length*0.8):]
testneg = traindf[traindf['sentiment']==0].iloc[int(neg_length*0.8):]
frames = [trainpos,trainneg]
traindata = pd.concat(frames)
frames = [testpos,testneg]
testdata = pd.concat(frames)
traindata['review_text'] = pd.Series(traindata.text,index = traindata.index)
testdata['review_text'] = pd.Series(testdata.text,index = testdata.index)

39691 18577


In [123]:
# Unigram Set:
start_time  = time.time()
fdist = nltk.FreqDist()
for i in traindata.index:
    ly = traindata.text[i].lower()
    ly = ly.decode('utf-8')
    #1.Tokenization:
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        continue
        
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]

    fdist = fdist.__add__(nltk.FreqDist(word_stop))
    
print("--- %s seconds ---" % (time.time() - start_time))

fdist_uni = fdist

--- 7787.71427894 seconds ---


In [123]:
"""
ly = ly.decode('utf-8')
#1.Tokenization:
tokens = nltk.word_tokenize(ly)
#2.Special Characters Removed:
tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
tokens_lem_stem = []
for word in tokens:
    try:
        #3.First Lemmatization, 4.then Stemming:
        tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
    except UnicodeDecodeError:
        continue

#5.Remove Punctuation:
word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
#6.Remove stop words:
word_stop = [w for w in word_punct if w not in stopwords.words("english")]

final_list = word_stop
"""

In [124]:
num_unigram = 5000
# create features for training data
feat_uni = [i for [i,j] in fdist_uni.most_common(num_unigram)]
for unigram in feat_uni:
    traindata[unigram] = pd.Series([0.0]*len(traindata),index = traindata.index)
#create bigram features for training set:
start_time = time.time()
#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in traindata.index:
    ly = traindata.review_text[i].lower()
    
    ly = ly.decode('utf-8')
    #1.Tokenization:
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        print ly
        continue
        
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]
    
    
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in feat_uni:
        traindata.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    traindata.set_value(i, unigram, 1)
print("--- %s seconds ---" % (time.time() - start_time))




#change features for test data
for unigram in feat_uni:
    testdata[unigram] = pd.Series([0.0]*len(testdata),index = testdata.index)

start_time = time.time()
#Try to use unigram to analyze and obtain the feature:
fdist = nltk.FreqDist()
for i in testdata.index:
    ly = testdata.review_text[i].lower()
    ly = ly.decode('utf-8')
    #1.Tokenization:
    try:
        tokens = nltk.word_tokenize(ly)
    except UnicodeDecodeError:
        print ly
        break
        
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]    
    
    
    #compute frequency distribution for all the unigram in one song:
    fdist = nltk.FreqDist(word_stop)
    for unigram in feat_uni:
        testdata.set_value(i, unigram, fdist[unigram])
        #if unigram in word_stop:
        #    testdata.set_value(i, unigram, 1)    
print("--- %s seconds ---" % (time.time() - start_time))

--- 2299.18544507 seconds ---
--- 575.722703934 seconds ---


In [125]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 500,criterion="entropy",oob_score=True)#,min_samples_leaf=15)
clf = clf.fit(traindata[feat_uni],traindata['sentiment'])

print 'Correct rate of training set:',clf.score(traindata[feat_uni],traindata['sentiment'])
print 'Correct oob rate of training set:',clf.oob_score_
print 'Correct rate of test set:',clf.score(testdata[feat_uni],testdata['sentiment'])


"""
res = clf.predict(testdata[feat_uni])
for i in set(testdata.sentiment.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdata['sentiment'])==i)/float(len(res))
"""

print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.999957093515
Correct oob rate of training set: 0.922725419947
Correct rate of test set: 0.922951522952
Results of test set with 0 decades difference : 0.922951522952
Results of test set with 1 decades difference : 0.0770484770485
--- 1464.67448497 seconds ---


In [129]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 100,criterion="entropy",oob_score=True,min_samples_leaf=100)
clf = clf.fit(traindata[feat_uni],traindata['sentiment'])

print 'Correct rate of training set:',clf.score(traindata[feat_uni],traindata['sentiment'])
print 'Correct oob rate of training set:',clf.oob_score_
print 'Correct rate of test set:',clf.score(testdata[feat_uni],testdata['sentiment'])

"""
res = clf.predict(testdata[feat_uni])
for i in set(testdata.sentiment.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdata['sentiment'])==i)/float(len(res))

"""
print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.823997597237
Correct oob rate of training set: 0.819599682492
Correct rate of test set: 0.823852423852
Results of test set with 0 decades difference : 0.823852423852
Results of test set with 1 decades difference : 0.176147576148
--- 151.296142101 seconds ---


In [128]:
start_time = time.time()

clf = RandomForestClassifier(n_estimators= 500,criterion="entropy",oob_score=True,min_samples_leaf=15)
clf = clf.fit(traindata[feat_uni],traindata['sentiment'])

print 'Correct rate of training set:',clf.score(traindata[feat_uni],traindata['sentiment'])
print 'Correct oob rate of training set:',clf.oob_score_
print 'Correct rate of test set:',clf.score(testdata[feat_uni],testdata['sentiment'])


"""
res = clf.predict(testdata[feat_uni])
for i in set(testdata.sentiment.values):
    print 'Results of test set with '+str(i)+' decades difference :',sum(map(abs,res - testdata['sentiment'])==i)/float(len(res))
"""

print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of training set: 0.905498466093
Correct oob rate of training set: 0.891940016734
Correct rate of test set: 0.893264693265
Results of test set with 0 decades difference : 0.893264693265
Results of test set with 1 decades difference : 0.106735306735
--- 863.067273855 seconds ---


In [ ]:
#Methods 2: Naive Bayes:

In [164]:
start_time = time.time()

gnb = GaussianNB()
y_pred = gnb.fit(traindata[feat_uni],traindata['sentiment']).predict(testdata[feat_uni])

print "Correct rate of test set:", float((testdata['sentiment'].values == y_pred).sum())/len(testdata)

print("--- %s seconds ---" % (time.time() - start_time))

Correct rate of test set: 0.172543972544


In [ ]:
#Methods 3: SVM:

In [ ]:
start_time = time.time()

clf = svm.SVR()
clf.fit(traindata[feat_uni],traindata['sentiment']) 
clf.predict(testdata[feat_uni])

print("--- %s seconds ---" % (time.time() - start_time))